In [1]:
#Imported relevant and necessary libraries and data cleaning tools
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import hypertools as hyp
import plotly_express as px #plotly express is a library for drawing interactive figures
from glob import glob as lsdir
import os
import re
import datetime as dt

from sklearn import linear_model
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split

%matplotlib inline

/optnfs/el7/jupyterhub/envs/Psych81.09/lib/python3.6/site-packages/hypertools/plot/__init__.py:10: UserWarning: Could not switch backend to TkAgg.  This may impact performance of the plotting functions.
  warnings.warn('Could not switch backend to TkAgg.  This may impact performance of the plotting functions.')


In [2]:
#Code from Professor Manning to set up and read in the relevant UVLT data 
data_readers = {'xlsx': pd.read_excel, 'xls': pd.read_excel, 'dta': pd.read_stata}
get_extension = lambda x: x.split('.')[-1]


def read_data(datadir, readers):
    files = lsdir(os.path.join('..', datadir, '*'))
    readable_files = []
    data = []
    for f in files:
        ext = get_extension(f)
        if ext in readers.keys():
            readable_files.append(f)
            data.append(data_readers[ext](f))
    return readable_files, data


fnames, data = read_data('data', data_readers)

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [3]:
#A summary of the data files that are now read into the notebook
fnames

['../data/UVLTdata_individual.dta',
 '../data/UVLTdata_final.dta',
 '../data/Direct Mailing Analysis.xlsx',
 '../data/UVLTDataAnalysis.xls',
 '../data/TownLevelData.xlsx',
 '../data/TownLevelData.dta',
 '../data/CensusInfoUpperValley2015JH.xlsx']

In [4]:
#Renaming relevant columns in UVLT individual data to be more easily readable
names={'DeceasedDateYN' : 'Is the donor Deceased?',
       'U_Tot_Amt': 'Total Unrestricted Donations',
      'U_Tot_Cnt': 'Total # Unrestricted Donations',
      'ConservedOwner' : 'Owns Conserved Land?',
      'RTotAmt' : 'Total Restricted Donations',
       'RTotCnt': 'Total # Restricted Donations',
      'VTotCnt' : 'Total Volunteer Occurances',
      'ETotCnt' : 'Total Event Attendances'}
data[1].rename(names, inplace=True, axis=1)

In [5]:
#copying each set of data into more memorably named versions
final_data=data[1].copy()

In [16]:
final_data.head()

,ContactID,State,TownID,Town,LandOwnerTownID,Is the donor Deceased?,Total Unrestricted Donations,Total # Unrestricted Donations,U200001,U200102,...,E201819,DeceasedDate,Owns Conserved Land?,Nprojects,Nacres,Nmembers,MedianHHIncome,MeanHHIncome,PercBAplus,PercAge55Plus
0,2903.0,VT,2.0,Bradford,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,0.0,20.0,2258.32,7.0,48056.0,58716.0,30.6,32.912306
1,11472.0,VT,2.0,Bradford,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,0.0,20.0,2258.32,7.0,48056.0,58716.0,30.6,32.912306
2,9206.0,VT,2.0,Bradford,0.0,0.0,75.0,2.0,0.0,0.0,...,0.0,NaN,0.0,20.0,2258.32,7.0,48056.0,58716.0,30.6,32.912306
3,12910.0,VT,2.0,Bradford,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,0.0,20.0,2258.32,7.0,48056.0,58716.0,30.6,32.912306
4,5029.0,VT,2.0,Bradford,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,0.0,20.0,2258.32,7.0,48056.0,58716.0,30.6,32.912306


In [6]:
#creates new DataFrame of average unrestricted donations -- this will be used to store all averages
avg_data = final_data.groupby('Town', as_index=False)['Total Unrestricted Donations'].mean()

In [7]:
#add columns with average town data to the new DataFrame
avg_data['Avg # UR Donations'] = final_data.groupby('Town', as_index=True)['Total # Unrestricted Donations'].mean().values
avg_data['Avg R Donations'] = final_data.groupby('Town', as_index=True)['Total Restricted Donations'].mean().values
avg_data['Avg # R Donations'] = final_data.groupby('Town', as_index=True)['Total # Restricted Donations'].mean().values
avg_data['Avg Volunteer Occurances'] = final_data.groupby('Town', as_index=True)['Total Volunteer Occurances'].mean().values
avg_data['Avg Event Attendances'] = final_data.groupby('Town', as_index=True)['Total Event Attendances'].mean().values
avg_data.rename({'Total Unrestricted Donations' : 'Avg UR Donations'}, inplace=True, axis = 1)

avg_data.drop(avg_data.index[0], inplace=True)

In [8]:
avg_data.head()

,Town,Avg UR Donations,Avg # UR Donations,Avg R Donations,Avg # R Donations,Avg Volunteer Occurances,Avg Event Attendances
1,All Other Towns,362.811579,0.769182,168.536277,0.114465,0.207547,0.267925
2,All Other Towns CT,442.218543,1.079470,350.066225,0.145695,0.000000,0.019868
3,All Other Towns MA,1250.132725,1.266247,689.746918,0.253669,0.041929,0.102725
4,All Other Towns ME,289.410364,1.945455,61.363636,0.163636,0.018182,0.236364
5,All Other Towns NH,96.395799,0.538970,465.202761,0.142668,0.129458,0.248349
6,All Other Towns VT,1228.228735,0.719368,595.466403,0.128458,0.191700,0.250988
7,Bath,NaN,NaN,NaN,NaN,NaN,NaN
8,Bradford,45.607143,0.878571,407.857143,0.035714,0.164286,0.457143
9,Canaan,90.153541,1.009569,3.110048,0.014354,0.272727,0.315789
10,Cavendish,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
zipcode_data=data[3].copy()
town_data=data[4].copy()

In [10]:
town_data.head()

,TownID,Town,Nprojects,Nacres,Nmembers,MedianHHIncome,MeanHHIncome,PercBAplus,PercAge55Plus
0,1,Bath,9,746.00,0,47386,60413,25.274725,48.844538
1,2,Bradford,20,2258.32,7,48056,58716,30.600000,32.912306
2,3,Canaan,9,1676.35,10,58333,68870,25.300000,33.863520
3,4,Cavendish,0,0.00,0,48750,69230,31.001727,40.359043
4,5,Charlestown,2,408.90,2,42693,50823,14.300000,36.050905


In [11]:
#merge town demographics to main averages DataFrame
avg_data = pd.merge(avg_data, town_data, on="Town")
avg_data.drop(['TownID'], axis=1)
avg_data.head()

,Town,Avg UR Donations,Avg # UR Donations,Avg R Donations,Avg # R Donations,Avg Volunteer Occurances,Avg Event Attendances,Nprojects,Nacres,Nmembers,MedianHHIncome,MeanHHIncome,PercBAplus,PercAge55Plus
0,Bath,NaN,NaN,NaN,NaN,NaN,NaN,9,746.00,0,47386,60413,25.274725,48.844538
1,Bradford,45.607143,0.878571,407.857143,0.035714,0.164286,0.457143,20,2258.32,7,48056,58716,30.600000,32.912306
2,Canaan,90.153541,1.009569,3.110048,0.014354,0.272727,0.315789,9,1676.35,10,58333,68870,25.300000,33.863520
3,Cavendish,NaN,NaN,NaN,NaN,NaN,NaN,0,0.00,0,48750,69230,31.001727,40.359043
4,Charlestown,22.196262,0.308411,5.607477,0.018692,0.093458,0.214953,2,408.90,2,42693,50823,14.300000,36.050905
5,Chelsea,11.833333,0.216667,0.000000,0.000000,0.450000,0.116667,5,911.50,1,58333,63588,30.000000,37.884471
6,Claremont,28.274390,0.189024,26.524390,0.048780,0.073171,0.237805,5,564.51,0,45510,55958,18.600000,27.844907
7,Corinth,51.086957,0.423913,0.000000,0.000000,0.010870,0.108696,19,2349.08,3,56719,71381,27.800000,33.715596
8,Cornish,106.832461,1.188482,174.057592,0.178010,0.062827,0.240838,22,1124.29,11,75417,82487,38.800000,38.562874
9,Croydon,NaN,NaN,NaN,NaN,NaN,NaN,1,26.00,0,71094,79616,16.200000,36.054422


In [14]:
#px.scatter(avg_data, x="Avg UR Donations", y="Avg R Donations", color="Town", log_x=True)